This notebook generates the processed data for dataset ds005, which is used in some of the chapter-specific notebooks from Poldrack, Mumford, and Nichols' _Handbook of fMRI Data Analysis (2nd Edition)_.  This also provides an example of using the nipype workflow mechanism.

In [1]:
import os, errno, sys,shutil
import json


from fmrihandbook.utils.config import Config

config=Config()

%matplotlib inline

from nipype.interfaces import fsl, nipy, ants
import nibabel
from wand.image import Image as WImage
import numpy
import nilearn.plotting
import matplotlib.pyplot as plt
from nilearn.input_data import NiftiMasker
import fmrihandbook.utils
from fmrihandbook.utils.compute_fd_dvars import compute_fd,compute_dvars
import pickle
from fmrihandbook.utils.get_data import get_data

import nipype.interfaces.io as nio           # Data i/o
import nipype.pipeline.engine as pe          # pypeline engine
import nipype.algorithms.modelgen as model   # model specification
from nipype.interfaces.base import Bunch
import glob
import nipype.interfaces.utility as niu
from nipype.interfaces.c3 import C3dAffineTool
from nipype.interfaces.utility import Merge, IdentityInterface


rerun_analyses=False  # set to true to force rerun of everything



/Users/poldrack/anaconda/envs/py3k/lib/python3.5/importlib/_bootstrap_external.py:903: FutureWarning: Module nipy.labs.utils.routines deprecated, will be removed
  _imp.create_dynamic, spec)
/Users/poldrack/anaconda/envs/py3k/lib/python3.5/site-packages/nipype/interfaces/nipy/model.py:18: FutureWarning: Module nipy.labs.glm deprecated, will be removed. Please use nipy.modalities.fmri.glm instead.
  import nipy.labs.glm.glm as GLM


using base dir: /Users/poldrack/data/fmri-handbook-2e-data


In [2]:
config.data=get_data('ds005')
print(config.data)

{'ds031': {'T2': '/Users/poldrack/data/fmri-handbook-2e-data/ds031/sub-01/ses-018/anat/sub-01_ses-018_run-001_T2w.nii.gz', 'func': '/Users/poldrack/data/fmri-handbook-2e-data/ds031/sub-01/ses-014/func/sub-01_ses-014_task-nback_run-001_bold.nii.gz', 'datadir': '/Users/poldrack/data/fmri-handbook-2e-data/ds031', 'fieldmap-phasediff': '/Users/poldrack/data/fmri-handbook-2e-data/ds031/sub-01/ses-014/fmap/sub-01_ses-014_phasediff.nii.gz', 'T1': '/Users/poldrack/data/fmri-handbook-2e-data/ds031/sub-01/ses-018/anat/sub-01_ses-018_run-001_T1w.nii.gz', 'fieldmap-mag': '/Users/poldrack/data/fmri-handbook-2e-data/ds031/sub-01/ses-014/fmap/sub-01_ses-014_magnitude1.nii.gz', 'sbref': '/Users/poldrack/data/fmri-handbook-2e-data/ds031/sub-01/ses-014/func/sub-01_ses-014_task-nback_run-001_sbref.nii.gz'}, 'ds005': {'datadir': '/Users/poldrack/data/fmri-handbook-2e-data/ds005'}}


## Structural workflow

In [3]:

subcodes=[os.path.basename(i) for i in glob.glob(os.path.join(config.data['ds005']['datadir'],'sub-*'))]
subcodes.sort()
#subcodes=subcodes[:1]
print(subcodes)

# Map field names to individual subject runs.
info = dict(func=[['subject_id','subject_id','runcode']],
            anat=[['subject_id', 'subject_id']])

infosource = pe.Node(interface=niu.IdentityInterface(fields=['subject_id']), name="infosource")

# this builds off of example at http://www.mit.edu/~satra/nipype-nightly/users/examples/fmri_ants_openfmri.html

infosource.iterables = ('subject_id', subcodes)

runinfo = pe.Node(interface=niu.IdentityInterface(fields=['runcode']), name="runinfo")

runinfo.iterables = ('runcode',['1','2','3'])


datasource_anat = pe.Node(interface=nio.DataGrabber(infields=['subject_id'],
                    outfields=['anat']),
                    name = 'datasource_anat')

datasource_anat.inputs.base_directory = config.data['ds005']['datadir']

datasource_anat.inputs.template = '%s/anat/%s_T1w.nii.gz'


datasource_anat.inputs.template_args = dict(anat=[['subject_id','subject_id']])

datasource_anat.inputs.sort_filelist = True

                     
datasource_func = pe.Node(interface=nio.DataGrabber(infields=['subject_id','runcode'],
                    outfields=['func']),
                    name = 'datasource_func')

datasource_func.inputs.base_directory = config.data['ds005']['datadir']

datasource_func.inputs.template = '%s/func/%s_task-mixedgamblestask_run-0%s_bold.nii.gz'

datasource_func.inputs.template_args = dict(func=[['subject_id','subject_id','runcode']])

datasource_func.inputs.sort_filelist = True

                                                 
preprocessing = pe.Workflow(name="preprocessing")

try:
    workdir='/Users/poldrack/data_unsynced/fmri-handbook-2e-data/ds005/nipype_workdir'
    assert os.path.exists(workdir)
except:
    workdir=os.path.join(config.data['ds005']['datadir'],'nipype_workdir')
    if not os.path.exists(workdir):
        os.mkdir(workdir)

preprocessing.base_dir = workdir

preprocessing.connect(infosource,'subject_id',datasource_anat,'subject_id')

preprocessing.connect(infosource,'subject_id',datasource_func,'subject_id')
preprocessing.connect(runinfo,'runcode',datasource_func,'runcode')



['sub-01', 'sub-02', 'sub-03', 'sub-04', 'sub-05', 'sub-06', 'sub-07', 'sub-08', 'sub-09', 'sub-10', 'sub-11', 'sub-12', 'sub-13', 'sub-14', 'sub-15', 'sub-16']


### Bias field correction

In [4]:
bfc = pe.Node(interface=ants.N4BiasFieldCorrection(), name="bfc")
bfc.inputs.dimension = 3
bfc.inputs.save_bias = True

preprocessing.connect(datasource_anat, 'anat', bfc, 'input_image')

datasink = pe.Node(nio.DataSink(), name='datasink')

# Save the relevant data into an output directory
datasink.inputs.base_directory = os.path.join(config.data['ds005']['datadir'],'derivatives')
if not os.path.exists(datasink.inputs.base_directory):
    os.mkdir(datasink.inputs.base_directory)


preprocessing.connect(bfc, 'bias_image', datasink, 'bfc.bias')
preprocessing.connect(bfc, 'output_image', datasink, 'bfc.output')


### Brain extraction using BET###

In [5]:
bet_struct=pe.Node(interface=fsl.BET(), name="bet_struct")
bet_struct.inputs.reduce_bias=True
bet_struct.inputs.frac=0.4

preprocessing.connect(bfc,'output_image',bet_struct,'in_file')
preprocessing.connect(bet_struct, 'out_file', datasink, 'bet.output')
preprocessing.connect(bet_struct, 'mask_file', datasink, 'bet.mask')



### Segmentation using FAST

Do this to obtain the white matter mask, which we need for BBR registration.

In [6]:
fast=pe.Node(interface=fsl.FAST(), name="fast")

preprocessing.connect(bet_struct,'out_file',fast,'in_files')
preprocessing.connect(fast, 'partial_volume_files', datasink, 'fast.pvefiles')
preprocessing.connect(fast, 'tissue_class_map', datasink, 'fast.seg')

binarize = pe.Node(fsl.ImageMaths(op_string='-nan -thr 0.5 -bin'),
                   name='binarize')
pickindex = lambda x, i: x[i]
preprocessing.connect(fast, ('partial_volume_files', pickindex, 2),
                 binarize, 'in_file')
preprocessing.connect(binarize, 'out_file', datasink, 'fast.wmseg')



### Spatial normalization using ANTs

In [7]:
antsreg=pe.Node(interface=ants.Registration(), name="antsreg")
antsreg.inputs.fixed_image = os.path.join(os.getenv('FSLDIR'),'data/standard/MNI152_T1_2mm_brain.nii.gz')
antsreg.inputs.transforms = ['Translation', 'Rigid', 'Affine', 'SyN']
antsreg.inputs.transform_parameters = [(0.1,), (0.1,), (0.1,), (0.2, 3.0, 0.0)]
antsreg.inputs.number_of_iterations = ([[10, 10, 10]]*3 +
                [[1, 5, 3]])
antsreg.inputs.dimension = 3
antsreg.inputs.write_composite_transform = True
antsreg.inputs.metric = ['Mattes'] * 3 + [['Mattes', 'CC']]
antsreg.inputs.metric_weight = [1] * 3 + [[0.5, 0.5]]
antsreg.inputs.radius_or_number_of_bins = [32] * 3 + [[32, 4]]
antsreg.inputs.sampling_strategy = ['Regular'] * 3 + [[None, None]]
antsreg.inputs.sampling_percentage = [0.3] * 3 + [[None, None]]
antsreg.inputs.convergence_threshold = [1.e-8] * 3 + [-0.01]
antsreg.inputs.convergence_window_size = [20] * 3 + [5]
antsreg.inputs.smoothing_sigmas = [[4, 2, 1]] * 3 + [[1, 0.5, 0]]
antsreg.inputs.sigma_units = ['vox'] * 4
antsreg.inputs.shrink_factors = [[6, 4, 2]] + [[3, 2, 1]]*2 + [[4, 2, 1]]
antsreg.inputs.use_estimate_learning_rate_once = [True] * 4
antsreg.inputs.use_histogram_matching = [False] * 3 + [True]
antsreg.inputs.initial_moving_transform_com = True
antsreg.inputs.output_warped_image = True

preprocessing.connect(bet_struct,'out_file',antsreg,'moving_image')
preprocessing.connect(antsreg, 'warped_image', datasink, 'ants.warped_image')
preprocessing.connect(antsreg, 'composite_transform', datasink, 'ants.composite_transform')
preprocessing.connect(antsreg, 'inverse_composite_transform', datasink, 'ants.inverse_composite_transform')




In [8]:
# get linear warp of anatomy to MNI space, for comparison to nonlinear
linearMNI=pe.Node(fsl.FLIRT(), name='linearMNI')
linearMNI.inputs.reference=os.path.join(os.getenv('FSLDIR'),'data/standard/MNI152_T1_2mm_brain.nii.gz')
linearMNI.inputs.dof=12

preprocessing.connect(bet_struct,'out_file',linearMNI,'in_file')

preprocessing.connect(linearMNI,'out_file',datasink,'affine.out_file')
preprocessing.connect(linearMNI,'out_matrix_file',datasink,'affine.matrix')




## Functional preprocessing

### Motion correction using MCFLIRT

This will take a few minutes.

In [9]:
mcflirt=pe.Node(interface=fsl.MCFLIRT(), name="mcflirt")
mcflirt.inputs.save_plots=True
mcflirt.inputs.mean_vol=True

preprocessing.connect(datasource_func, 'func', mcflirt, 'in_file')

preprocessing.connect(mcflirt, 'out_file', datasink, 'mcflirt.out_file')
preprocessing.connect(mcflirt, 'par_file', datasink, 'mcflirt.par')
preprocessing.connect(mcflirt, 'mean_img', datasink, 'mcflirt.mean')




Make links for the mean functional image and the motion parameters.

## Brain extraction

Use FSL's BET to obtain the brain mask for the functional data

In [10]:
bet_func=pe.Node(interface=fsl.BET(), name="bet_func")

bet_func.inputs.functional=True
bet_func.inputs.mask=True

preprocessing.connect(mcflirt, 'out_file', bet_func, 'in_file')

preprocessing.connect(bet_func, 'out_file', datasink, 'betfunc.out_file')
preprocessing.connect(bet_func, 'mask_file', datasink, 'betfunc.mask_file')

meanbetfunc=pe.Node(interface=fsl.MeanImage(), name="meanbetfunc")
preprocessing.connect(bet_func,'out_file',meanbetfunc,'in_file')
preprocessing.connect(meanbetfunc, 'out_file', datasink, 'betfunc.mean_file')



#### BBR registration of functional to structural

In [11]:
mean2anat = pe.Node(fsl.FLIRT(), name='mean2anat')
mean2anat.inputs.dof = 6
preprocessing.connect(meanbetfunc, 'out_file', mean2anat, 'in_file')
preprocessing.connect(bet_struct, 'out_file', mean2anat, 'reference')

mean2anatbbr = pe.Node(fsl.FLIRT(), name='mean2anatbbr')
mean2anatbbr.inputs.dof = 6
mean2anatbbr.inputs.cost = 'bbr'
mean2anatbbr.inputs.schedule = os.path.join(os.getenv('FSLDIR'),
                                            'etc/flirtsch/bbr.sch')

preprocessing.connect(meanbetfunc, 'out_file', mean2anatbbr, 'in_file')
preprocessing.connect(binarize, 'out_file', mean2anatbbr, 'wm_seg')
preprocessing.connect(bet_struct, 'out_file', mean2anatbbr, 'reference')
preprocessing.connect(mean2anat, 'out_matrix_file',
                 mean2anatbbr, 'in_matrix_file')

preprocessing.connect(mean2anat, 'out_matrix_file', datasink, 'mean2anat.out_matrix')

preprocessing.connect(mean2anatbbr, 'out_matrix_file', datasink, 'bbr.out_matrix')
preprocessing.connect(mean2anatbbr, 'out_file', datasink, 'bbr.out_file')


# convert BBR matrix to ITK for ANTS

convert2itk = pe.Node(C3dAffineTool(),
                      name='convert2itk')
convert2itk.inputs.fsl2ras = True
convert2itk.inputs.itk_transform = True
preprocessing.connect(mean2anatbbr, 'out_matrix_file', convert2itk, 'transform_file')
preprocessing.connect(meanbetfunc, 'out_file', convert2itk, 'source_file')
preprocessing.connect(bet_struct, 'out_file', convert2itk, 'reference_file')



In [12]:
# Concatenate the affine and ants transforms into a list

pickfirst = lambda x: x[0]

merge = pe.Node(Merge(2),  name='mergexfm')
preprocessing.connect(convert2itk, 'itk_transform', merge, 'in2')
preprocessing.connect(antsreg, 'composite_transform', merge, 'in1')

warpmean = pe.Node(ants.ApplyTransforms(), name='warpmean')
warpmean.inputs.input_image_type = 0
warpmean.inputs.interpolation = 'Linear'
warpmean.inputs.invert_transform_flags = [False] #[False, False]
warpmean.inputs.terminal_output = 'file'
warpmean.inputs.args = '--float'
warpmean.inputs.reference_image = os.path.join(os.getenv('FSLDIR'),'data/standard/MNI152_T1_2mm_brain.nii.gz')


preprocessing.connect(mean2anatbbr, 'out_file', warpmean, 'input_image')
preprocessing.connect(antsreg, 'composite_transform', warpmean, 'transforms')
#preprocessing.connect(merge, 'out', warpmean, 'transforms')
preprocessing.connect(warpmean,'output_image',datasink,'ants.warped_mean')


# high pass filtering

hpfilt=pe.Node(interface=fsl.maths.TemporalFilter(),name='hpfilt')
TR=2.0
hpf_cutoff=80.

hpfilt.inputs.highpass_sigma = hpf_cutoff/(2*TR)
preprocessing.connect(bet_func, 'out_file',hpfilt,'in_file')


preprocessing.connect(hpfilt,'out_file',datasink,'hpfilt')

rescale = pe.Node(interface=fsl.maths.BinaryMaths(),name='rescale')
rescale.inputs.operation = "add"

preprocessing.connect(hpfilt,'out_file',rescale,'in_file')
preprocessing.connect(meanbetfunc, 'out_file',rescale,'operand_file')

preprocessing.connect(rescale,'out_file',datasink,'rescaled')

# spatial smoothing using FSL's susan or regular gaussian smoothing

use_SUSAN=False

if use_SUSAN:
    medianval = pe.MapNode(interface=fsl.ImageStats(op_string='-k %s -p 50'),
                           iterfield=['in_file','mask_file'],
                           name='medianval')
    preprocessing.connect(mcflirt, 'out_file', medianval, 'in_file')
    preprocessing.connect(bet_func, 'mask_file', medianval, 'mask_file')

    smooth = pe.MapNode(interface=fsl.SUSAN(),
                        iterfield=['in_file', 'brightness_threshold'],
                        name='smooth')
    def getbtthresh(medianvals):
        return [0.75 * val for val in medianvals]

    preprocessing.connect(medianval, ('out_stat', getbtthresh), smooth, 'brightness_threshold')
else:
    smooth=pe.Node(interface=fsl.utils.Smooth(), name="smooth")

smooth.inputs.fwhm=6

# Turn off smoothing for now - we will do it later on the constrast images
#preprocessing.connect(rescale,'out_file',smooth,'in_file')
#preprocessing.connect(smooth,'smoothed_file',datasink,'smooth')



graph=preprocessing.run() #plugin='MultiProc', plugin_args={'n_procs' : 2})




/Users/poldrack/anaconda/envs/py3k/lib/python3.5/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)
/Users/poldrack/anaconda/envs/py3k/lib/python3.5/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)
INFO:workflow:['check', 'execution', 'logging']
INFO:workflow:Running serially.
INFO:workflow:Executing node datasource_func.a2.a08 in dir: /Users/poldrack/data_unsynced/fmri-handbook-2e-data/ds005/nipype_workdir/preprocessing/_runcode_3/_subject_id_sub-09/datasource_func
INFO:workflow:Executing node datasource_func.a2.a00 in dir: /Users/poldrack/data_unsynced/fmri-handbook-2e-data/ds005/nipype_workdir/preprocessing/_runcode_3/_subject_id_sub-

### Registration check
#### Show overlays of highres-template and functional-highres

In [13]:
regcheck=False
if regcheck:
  for subcode in subcodes:
    antsregfile=os.path.join(config.data['ds005']['datadir'],
                             'derivatives/ants/warped_image/_subject_id_%s/transform_Warped.nii.gz'%subcode)
    if os.path.exists(antsregfile):
        mask_display=nilearn.plotting.plot_epi(antsregfile,cmap='gray')
        mask_display.add_edges(os.path.join(os.getenv('FSLDIR'),'data/standard/MNI152_T1_2mm_brain.nii.gz'))

In [14]:
# linear warp
if regcheck:
  for subcode in subcodes:
    antsregfile=os.path.join(config.data['ds005']['datadir'],
                             'derivatives/affine/out_file/_subject_id_%s/%s_T1w_corrected_brain_flirt.nii.gz'%(subcode,subcode))
    if os.path.exists(antsregfile):
        mask_display=nilearn.plotting.plot_epi(antsregfile,cmap='gray')
        mask_display.add_edges(os.path.join(os.getenv('FSLDIR'),'data/standard/MNI152_T1_2mm_brain.nii.gz'))

In [15]:
if regcheck:
  for subcode in subcodes:
    antsregfile=os.path.join(config.data['ds005']['datadir'],
                             'derivatives/bet/output/_subject_id_%s/%s_T1w_corrected_brain.nii.gz'%(subcode,subcode))
    if os.path.exists(antsregfile):
        mask_display=nilearn.plotting.plot_epi(antsregfile,cmap='gray')
        mask_display.add_edges(os.path.join(config.data['ds005']['datadir'],
            'derivatives/bbr/out_file/_runcode_1/_subject_id_%s/%s_task-mixedgamblestask_run-01_bold_mcf_brain_mean_flirt.nii.gz'%(subcode,subcode)))

In [16]:
# warped functional to MNI
if regcheck:
  for subcode in subcodes:
    antsregfile=os.path.join(config.data['ds005']['datadir'],
       'derivatives/ants/warped_mean/_runcode_1/_subject_id_%s/%s_task-mixedgamblestask_run-01_bold_mcf_brain_mean_flirt_trans.nii.gz'%(subcode,subcode))
    if os.path.exists(antsregfile):
        mask_display=nilearn.plotting.plot_epi(antsregfile,cmap='gray')
        mask_display.add_edges(os.path.join(os.getenv('FSLDIR'),'data/standard/MNI152_T1_2mm_brain.nii.gz'))

### QA for fMRI data

Load motion data and compute FD/DVARS

In [17]:
do_qa=False
if do_qa:
    fd={}
    dvars={}
    for subcode in subcodes:
        fd[subcode]={}
        dvars[subcode]={}
        plt.figure(figsize=(12,4))
        for runcode in range(1,4):
            motfile=os.path.join(config.data['ds005']['datadir'],
                'derivatives/mcflirt/par/_runcode_%d/_subject_id_%s/%s_task-mixedgamblestask_run-%02d_bold_mcf.nii.gz.par'%(runcode,subcode,subcode,runcode))
            outdir=os.path.dirname(motfile)
            try:
                fd[subcode][runcode]=numpy.loadtxt(os.path.join(outdir,'fd.txt'))
                dvars[subcode][runcode]=numpy.loadtxt(os.path.join(outdir,'dvars.txt'))
            except:
                motiondata=numpy.loadtxt(motfile)
                fd[subcode][runcode]=compute_fd(motiondata)
                meanfile=os.path.join(config.data['ds005']['datadir'],
                    'derivatives/betfunc/out_file/_runcode_%d/_subject_id_%s/%s_task-mixedgamblestask_run-%02d_bold_mcf_brain.nii.gz'%(runcode,subcode,subcode,runcode))
                meandata=nibabel.load(meanfile)
                mask=nibabel.load(os.path.join(config.data['ds005']['datadir'],
                    'derivatives/betfunc/mask_file/_runcode_%d/_subject_id_%s/%s_task-mixedgamblestask_run-%02d_bold_mcf_brain_mask.nii.gz'%(runcode,subcode,subcode,runcode)))
                masker=NiftiMasker(mask_img=mask)
                maskdata=masker.fit_transform(meandata)
                globalmean=numpy.mean(maskdata,0)
                dvars[subcode][runcode]=compute_dvars(globalmean)
                numpy.savetxt(os.path.join(outdir,'fd.txt'),fd[subcode][runcode])
                numpy.savetxt(os.path.join(outdir,'dvars.txt'),dvars[subcode][runcode])

            plt.subplot(1,3,runcode)
            plt.plot(fd[subcode][runcode])
            #plt.plot(dvars[subcode][runcode]/100,color='r')

            plt.axis([0,250,0,2.0])
            plt.title('%s: run %d'%(subcode,runcode))


## First-level modeling

In [18]:
firstlevel = pe.Workflow(name="firstlevel")


firstlevel.base_dir = workdir


                     
datasource_func = pe.Node(interface=nio.DataGrabber(infields=['subject_id','runcode'],
                    outfields=['func']),
                    name = 'datasource_func')

datasource_func.inputs.base_directory = config.data['ds005']['datadir']

datasource_func.inputs.template = 'derivatives/rescaled/_runcode_%s/_subject_id_%s/%s_task-mixedgamblestask_run-0%s_bold_mcf_brain_filt_maths.nii.gz'

datasource_func.inputs.template_args = dict(func=[['runcode','subject_id','subject_id','runcode']])

datasource_func.inputs.sort_filelist = True


firstlevel.connect(infosource,'subject_id',datasource_anat,'subject_id')
firstlevel.connect(infosource,'subject_id',datasource_func,'subject_id')
firstlevel.connect(runinfo,'runcode',datasource_func,'runcode')




In [19]:
# creat a function that can read all of the onsets and motion files
# and generate the appropriate structure

def get_onsets(subject_id,runnum):
    from fmrihandbook.utils.config import Config
    from nipype.interfaces.base import Bunch
    import numpy
    import os,json


    config=Config()
    
    regressors=[]
    regressor_names=[]
    fdfile=os.path.join(config.data['ds005']['datadir'],'derivatives/mcflirt/par/_runcode_%s/_subject_id_%s/fd.txt'%(runnum,subject_id))
    regressors.append(numpy.loadtxt(fdfile))
    regressor_names.append('fd')
    
    dvarsfile=os.path.join(config.data['ds005']['datadir'],'derivatives/mcflirt/par/_runcode_%s/_subject_id_%s/dvars.txt'%(runnum,subject_id))
    regressors.append(numpy.loadtxt(dvarsfile))
    regressor_names.append('dvars')

    motfile=os.path.join(config.data['ds005']['datadir'],
                'derivatives/mcflirt/par/_runcode_%s/_subject_id_%s/%s_task-mixedgamblestask_run-0%s_bold_mcf.nii.gz.par'%(runnum,subject_id,subject_id,runnum))
    motpars=numpy.loadtxt(motfile)
    for i in range(motpars.shape[1]):
        regressors.append(motpars[:,i])
        regressor_names.append('motpar%d'%int(i+1))
        td=numpy.zeros(motpars.shape[0])
        td[1:]=motpars[1:,i] -motpars[:-1,i]
        regressors.append(td)
        regressor_names.append('disp%d'%int(i+1))

                         
    onsfile=os.path.join(config.data['ds005']['datadir'],'ds005_onsets.json')
    with open(onsfile, 'r') as f:
        ons = json.load(f) 
    conditions=['task','param-gain','param-loss','param-rt']
    info=ons[subject_id]['run-0%s'%runnum]
    onsets=[]
    durations=[]
    amplitudes=[]
    for c in conditions:
        onsets.append(info[c]['onset'])
        durations.append(info[c]['duration'])
        amplitudes.append(info[c]['weight'])
    
    info = [Bunch(conditions=conditions,
              onsets=onsets,
              durations=durations,
                amplitudes=amplitudes,
                 regressors=regressors,
                 regressor_names=regressor_names)]
    return info

getonsets = pe.Node(niu.Function(input_names=['subject_id','runnum'],
                output_names=['info'],
                function=get_onsets),
                name='getonsets')

firstlevel.connect(infosource,'subject_id',getonsets,'subject_id')
firstlevel.connect(runinfo,'runcode',getonsets,'runnum')

specifymodel=pe.Node(interface=model.SpecifyModel(),name='specifymodel')

specifymodel.inputs.input_units = 'secs'
#specifymodel.inputs.functional_runs = preprocessed_epi
specifymodel.inputs.time_repetition = 2.0
specifymodel.inputs.high_pass_filter_cutoff = hpf_cutoff
#s.inputs.subject_info = info
firstlevel.connect(getonsets,'info',specifymodel,'subject_info')
firstlevel.connect(datasource_func,'func',specifymodel,'functional_runs')



In [20]:
contrasts=[['task>Baseline','T', 
            ['task'],[1]],
           ['param-gain','T', 
            ['param-gain'],[1]],
           ['param-loss-neg','T', 
            ['param-loss'],[-1]],
           ['param-rt','T', 
            ['param-rt'],[1]]]
           
level1design = pe.Node(interface=fsl.model.Level1Design(),name='level1design')
level1design.inputs.interscan_interval =2.0
level1design.inputs.bases = {'dgamma':{'derivs': True}}
level1design.inputs.model_serial_correlations=True
level1design.inputs.contrasts=contrasts

firstlevel.connect(specifymodel,'session_info',level1design,'session_info')




In [21]:
modelgen = pe.Node(interface=fsl.model.FEATModel(),name='modelgen')
firstlevel.connect(level1design,'fsf_files',modelgen,'fsf_file')
firstlevel.connect(level1design,'ev_files',modelgen,'ev_files')

filmgls= pe.Node(interface=fsl.FILMGLS(),name='filmgls')

filmgls.inputs.autocorr_noestimate = True
firstlevel.connect(datasource_func,'func',filmgls,'in_file')
firstlevel.connect(modelgen,'design_file',filmgls,'design_file')
firstlevel.connect(modelgen,'con_file',filmgls,'tcon_file')

firstlevel.connect(filmgls,'param_estimates',datasink,'filmgls.param_estimates')
firstlevel.connect(filmgls,'sigmasquareds',datasink,'filmgls.sigmasquareds')
firstlevel.connect(filmgls,'copes',datasink,'filmgls.copes')
firstlevel.connect(filmgls,'varcopes',datasink,'filmgls.varcopes')
firstlevel.connect(filmgls,'dof_file',datasink,'filmgls.dof_file')
firstlevel.connect(filmgls,'tstats',datasink,'filmgls.tstats')
firstlevel.connect(filmgls,'zstats',datasink,'filmgls.zstats')


firstlevel.run()

INFO:workflow:['check', 'execution', 'logging']
INFO:workflow:Running serially.
INFO:workflow:Executing node datasource_func.a14.a2 in dir: /Users/poldrack/data_unsynced/fmri-handbook-2e-data/ds005/nipype_workdir/firstlevel/_runcode_3/_subject_id_sub-15/datasource_func
INFO:workflow:Executing node getonsets.a12.a2 in dir: /Users/poldrack/data_unsynced/fmri-handbook-2e-data/ds005/nipype_workdir/firstlevel/_runcode_3/_subject_id_sub-13/getonsets
INFO:workflow:Collecting precomputed outputs
INFO:workflow:Executing node datasource_func.a02.a2 in dir: /Users/poldrack/data_unsynced/fmri-handbook-2e-data/ds005/nipype_workdir/firstlevel/_runcode_3/_subject_id_sub-03/datasource_func
INFO:workflow:Executing node datasource_func.a15.a2 in dir: /Users/poldrack/data_unsynced/fmri-handbook-2e-data/ds005/nipype_workdir/firstlevel/_runcode_3/_subject_id_sub-16/datasource_func
INFO:workflow:Executing node datasource_func.a06.a2 in dir: /Users/poldrack/data_unsynced/fmri-handbook-2e-data/ds005/nipype_wo

## warp copes and varcopes to MNI space

use suboptimal two-step procedure - first apply linear registration from BBR, then apply nonlinear from ANTS

this is necesary because of errors getting FSL mat file into ANTS

In [22]:
datasource_stat = pe.Node(interface=nio.DataGrabber(infields=['subject_id','runcode'],
                    outfields=['stats']),
                    name = 'datasource_stat')

datasource_stat.inputs.base_directory = config.data['ds005']['datadir']
datasource_stat.inputs.template = 'derivatives/filmgls/*copes/_runcode_%s/_subject_id_%s/*.nii.gz'
datasource_stat.inputs.template_args = dict(stats=[['runcode','subject_id']])
datasource_stat.inputs.sort_filelist = True


datasource_bbrmat = pe.Node(interface=nio.DataGrabber(infields=['subject_id','runcode'],
                    outfields=['bbrmat']),
                    name = 'datasource_bbrmat')

datasource_bbrmat.inputs.base_directory = config.data['ds005']['datadir']
datasource_bbrmat.inputs.template = 'derivatives/bbr/out_matrix/_runcode_%s/_subject_id_%s/*.mat'
datasource_bbrmat.inputs.template_args = dict(bbrmat=[['runcode','subject_id']])
datasource_bbrmat.inputs.sort_filelist = True

datasource_antsreg = pe.Node(interface=nio.DataGrabber(infields=['subject_id','runcode'],
                    outfields=['composite_transform']),
                    name = 'datasource_antsreg')

datasource_antsreg.inputs.base_directory = config.data['ds005']['datadir']
datasource_antsreg.inputs.template = 'derivatives/ants/composite_transform/_subject_id_%s/transformComposite.h5'
datasource_antsreg.inputs.template_args = dict(composite_transform=[['subject_id']])
datasource_antsreg.inputs.sort_filelist = True

datasource_linearreg = pe.Node(interface=nio.DataGrabber(infields=['subject_id','runcode'],
                    outfields=['matrix']),
                    name = 'datasource_linearreg')

datasource_linearreg.inputs.base_directory = config.data['ds005']['datadir']
datasource_linearreg.inputs.template = 'derivatives/affine/matrix/_subject_id_%s/%s_T1w_corrected_brain_flirt.mat'
datasource_linearreg.inputs.template_args = dict(matrix=[['subject_id','subject_id']])
datasource_linearreg.inputs.sort_filelist = True




secondlevel = pe.Workflow(name='secondlevel')
secondlevel.base_dir = workdir


secondlevel.connect(infosource,'subject_id',datasource_bbrmat,'subject_id')
secondlevel.connect(infosource,'subject_id',datasource_stat,'subject_id')
secondlevel.connect(infosource,'subject_id',datasource_anat,'subject_id')
secondlevel.connect(infosource,'subject_id',datasource_antsreg,'subject_id')
secondlevel.connect(infosource,'subject_id',datasource_linearreg,'subject_id')
secondlevel.connect(infosource,'subject_id',datasource_func,'subject_id')




secondlevel.connect(runinfo,'runcode',datasource_stat,'runcode')
secondlevel.connect(runinfo,'runcode',datasource_bbrmat,'runcode')
secondlevel.connect(runinfo,'runcode',datasource_anat,'runcode')
secondlevel.connect(runinfo,'runcode',datasource_antsreg,'runcode')
secondlevel.connect(runinfo,'runcode',datasource_linearreg,'runcode')
secondlevel.connect(runinfo,'runcode',datasource_func,'runcode')


bbrstats=pe.MapNode(fsl.FLIRT(),name='bbrstats', iterfield=['in_file'])
bbrstats.inputs.apply_xfm=True


secondlevel.connect(datasource_stat, 'stats', bbrstats,'in_file')
secondlevel.connect(datasource_bbrmat, 'bbrmat', bbrstats, 'in_matrix_file')
secondlevel.connect(datasource_anat, 'anat', bbrstats,'reference')

secondlevel.connect(bbrstats,'out_file',datasink,'bbr.stats')



/Users/poldrack/anaconda/envs/py3k/lib/python3.5/site-packages/nipype/interfaces/base.py:432: UserWarning: Input apply_xfm requires inputs: in_matrix_file
  warn(msg)


In [23]:
# warp stats maps using both ANTS and linear registration

warpstats = pe.MapNode(ants.ApplyTransforms(), name='warpstats', iterfield=['input_image'])
warpstats.inputs.input_image_type = 0
warpstats.inputs.interpolation = 'Linear'
warpstats.inputs.invert_transform_flags = [False] #[False, False]
warpstats.inputs.terminal_output = 'file'
warpstats.inputs.args = '--float'
warpstats.inputs.reference_image = os.path.join(os.getenv('FSLDIR'),'data/standard/MNI152_T1_2mm_brain.nii.gz')

#secondlevel.connect(bbrstats, 'out_file', warpstats, 'subject_id')


secondlevel.connect(bbrstats, 'out_file', warpstats, 'input_image')
secondlevel.connect(datasource_antsreg, 'composite_transform', warpstats, 'transforms')
secondlevel.connect(warpstats,'output_image',datasink,'ants.warped_stats')

warpstats_linear=pe.MapNode(fsl.FLIRT(),name='warpstats_linear', iterfield=['in_file'])
warpstats_linear.inputs.apply_xfm=True
warpstats_linear.inputs.reference=os.path.join(os.getenv('FSLDIR'),'data/standard/MNI152_T1_2mm_brain.nii.gz')


secondlevel.connect(bbrstats, 'out_file', warpstats_linear, 'in_file')
secondlevel.connect(datasource_linearreg, 'matrix', warpstats_linear, 'in_matrix_file')
secondlevel.connect(warpstats_linear,'out_file',datasink,'affine.warped_stats')


secondlevel.run()

/Users/poldrack/anaconda/envs/py3k/lib/python3.5/site-packages/nipype/interfaces/base.py:432: UserWarning: Input apply_xfm requires inputs: in_matrix_file
  warn(msg)
INFO:workflow:['check', 'execution', 'logging']
INFO:workflow:Running serially.
INFO:workflow:Executing node datasource_bbrmat.a0.a15 in dir: /Users/poldrack/data_unsynced/fmri-handbook-2e-data/ds005/nipype_workdir/secondlevel/_runcode_1/_subject_id_sub-16/datasource_bbrmat
INFO:workflow:Executing node datasource_func.a1.a01 in dir: /Users/poldrack/data_unsynced/fmri-handbook-2e-data/ds005/nipype_workdir/firstlevel/_runcode_2/_subject_id_sub-02/datasource_func
INFO:workflow:Executing node datasource_stat.a1.a06 in dir: /Users/poldrack/data_unsynced/fmri-handbook-2e-data/ds005/nipype_workdir/secondlevel/_runcode_2/_subject_id_sub-07/datasource_stat
INFO:workflow:Executing node datasource_stat.a2.a12 in dir: /Users/poldrack/data_unsynced/fmri-handbook-2e-data/ds005/nipype_workdir/secondlevel/_runcode_3/_subject_id_sub-13/da

### Second level analysis - fixed effects across runs

In [24]:
fixed_fx = pe.Workflow(name='fixedfx')
fixed_fx.base_dir = workdir

copeinfo = pe.Node(interface=niu.IdentityInterface(fields=['copenum']), name="copeinfo")

copeinfo.iterables = ('copenum',[1,2,3,4])

regtypes = pe.Node(interface=niu.IdentityInterface(fields=['regtype']), name="regtypes")

regtypes.iterables = ('regtype',['ants','affine'])


datasource_cope = pe.Node(interface=nio.DataGrabber(infields=['subject_id','copenum','regtype'],
                    outfields=['copes']),
                    name = 'datasource_cope')

datasource_cope.inputs.base_directory = config.data['ds005']['datadir']
datasource_cope.inputs.template = 'derivatives/%s/warped_stats/_runcode_*/_subject_id_%s/_warpstats*/cope%d_*.nii.gz'
datasource_cope.inputs.template_args = dict(copes=[['regtype','subject_id','copenum']])
datasource_cope.inputs.sort_filelist = True

datasource_varcope = pe.Node(interface=nio.DataGrabber(infields=['subject_id','copenum','regtype'],
                    outfields=['varcopes']),
                    name = 'datasource_varcope')

datasource_varcope.inputs.base_directory = config.data['ds005']['datadir']
datasource_varcope.inputs.template = 'derivatives/%s/warped_stats/_runcode_*/_subject_id_%s/_warpstats*/varcope%d_*.nii.gz'
datasource_varcope.inputs.template_args = dict(varcopes=[['regtype','subject_id','copenum']])
datasource_varcope.inputs.sort_filelist = True


fixed_fx.connect(infosource,'subject_id',datasource_cope,'subject_id')
fixed_fx.connect(infosource,'subject_id',datasource_varcope,'subject_id')
fixed_fx.connect(copeinfo,'copenum',datasource_cope,'copenum')
fixed_fx.connect(copeinfo,'copenum',datasource_varcope,'copenum')
fixed_fx.connect(regtypes,'regtype',datasource_cope,'regtype')
fixed_fx.connect(regtypes,'regtype',datasource_varcope,'regtype')



copemerge    = pe.MapNode(interface=fsl.Merge(dimension='t'),
                          iterfield=['in_files'],
                          name="copemerge")

varcopemerge = pe.MapNode(interface=fsl.Merge(dimension='t'),
                       iterfield=['in_files'],
                       name="varcopemerge")

fixed_fx.connect(datasource_cope,'copes',copemerge,'in_files')
fixed_fx.connect(datasource_varcope,'varcopes',varcopemerge,'in_files')

level2model = pe.Node(interface=fsl.L2Model(),
                      name='l2model')
level2model.inputs.num_copes=3


flameo = pe.MapNode(interface=fsl.FLAMEO(run_mode='fe'), name="flameo",
                    iterfield=['cope_file','var_cope_file'])
flameo.inputs.mask_file=os.path.join(os.getenv('FSLDIR'),'data/standard/MNI152_T1_2mm_brain_mask.nii.gz')


fixed_fx.connect(copemerge,'merged_file',flameo,'cope_file')
fixed_fx.connect(varcopemerge,'merged_file',flameo,'var_cope_file')

fixed_fx.connect(level2model,'design_mat',flameo,'design_file')
fixed_fx.connect(level2model,'design_con',flameo,'t_con_file')
fixed_fx.connect(level2model,'design_grp',flameo,'cov_split_file')

fixed_fx.connect(flameo,'copes',datasink,'fixedfx.cope')
fixed_fx.connect(flameo,'var_copes',datasink,'fixedfx.varcope')


fixed_fx.run()



INFO:workflow:['check', 'execution', 'logging']
INFO:workflow:Running serially.
INFO:workflow:Executing node datasource_cope.a08.a1.a1 in dir: /Users/poldrack/data_unsynced/fmri-handbook-2e-data/ds005/nipype_workdir/fixedfx/_copenum_2/_regtype_affine/_subject_id_sub-09/datasource_cope
INFO:workflow:Executing node datasource_varcope.a15.a0.a1 in dir: /Users/poldrack/data_unsynced/fmri-handbook-2e-data/ds005/nipype_workdir/fixedfx/_copenum_1/_regtype_affine/_subject_id_sub-16/datasource_varcope
INFO:workflow:Executing node datasource_cope.a02.a2.a1 in dir: /Users/poldrack/data_unsynced/fmri-handbook-2e-data/ds005/nipype_workdir/fixedfx/_copenum_3/_regtype_affine/_subject_id_sub-03/datasource_cope
INFO:workflow:Executing node datasource_cope.a09.a3.a1 in dir: /Users/poldrack/data_unsynced/fmri-handbook-2e-data/ds005/nipype_workdir/fixedfx/_copenum_4/_regtype_affine/_subject_id_sub-10/datasource_cope
INFO:workflow:Executing node datasource_varcope.a12.a1.a1 in dir: /Users/poldrack/data_uns

### Third-level model: across subjects

In [25]:
mixed_fx = pe.Workflow(name='mixedfx')
mixed_fx.base_dir = workdir

smoothlevels = pe.Node(interface=niu.IdentityInterface(fields=['fwhm']), name="smoothlevels")

smoothlevels.iterables = ('fwhm',[0,4,8,16,32])



datasource_cope = pe.Node(interface=nio.DataGrabber(infields=['copenum','regtype'],
                    outfields=['copes']),
                    name = 'datasource_cope')

datasource_cope.inputs.base_directory = config.data['ds005']['datadir']
#fixedfx/cope/_copenum_1/_regtype_affine/_subject_id_sub-03/_flameo0/cope1.nii.gz
datasource_cope.inputs.template = 'derivatives/fixedfx/cope/_copenum_%d/_regtype_%s/_subject_id_*/_flameo0/cope1.nii.gz'
datasource_cope.inputs.template_args = dict(copes=[['copenum','regtype']])
datasource_cope.inputs.sort_filelist = True

datasource_varcope = pe.Node(interface=nio.DataGrabber(infields=['copenum','regtype'],
                    outfields=['varcopes']),
                    name = 'datasource_varcope')

datasource_varcope.inputs.base_directory = config.data['ds005']['datadir']
datasource_varcope.inputs.template = 'derivatives/fixedfx/varcope/_copenum_%d/_regtype_%s/_subject_id_*/_flameo0/varcope1.nii.gz'
datasource_varcope.inputs.template_args = dict(varcopes=[['copenum','regtype']])
datasource_varcope.inputs.sort_filelist = True


copemerge    = pe.MapNode(interface=fsl.Merge(dimension='t'),
                          iterfield=['in_files'],
                          name="copemerge")

varcopemerge = pe.MapNode(interface=fsl.Merge(dimension='t'),
                       iterfield=['in_files'],
                       name="varcopemerge")

mixed_fx.connect(copeinfo,'copenum',datasource_cope,'copenum')
mixed_fx.connect(copeinfo,'copenum',datasource_varcope,'copenum')
mixed_fx.connect(regtypes,'regtype',datasource_cope,'regtype')
mixed_fx.connect(regtypes,'regtype',datasource_varcope,'regtype')



mixed_fx.connect(datasource_cope,'copes',copemerge,'in_files')
mixed_fx.connect(datasource_varcope,'varcopes',varcopemerge,'in_files')

level3model = pe.Node(interface=fsl.L2Model(),
                      name='l3model')
level3model.inputs.num_copes=16

smooth_cope=pe.Node(interface=fsl.utils.Smooth(), name="smooth_cope",iterfield=['copenum','fwhm','regtype'])
smooth_varcope=pe.Node(interface=fsl.utils.Smooth(), name="smooth_varcope",iterfield=['copenum','fwhm','regtype'])


mixed_fx.connect(copemerge,('merged_file',pickfirst),smooth_cope,'in_file')
mixed_fx.connect(varcopemerge,('merged_file',pickfirst),smooth_varcope,'in_file')
mixed_fx.connect(smoothlevels,'fwhm',smooth_cope,'fwhm')
mixed_fx.connect(smoothlevels,'fwhm',smooth_varcope,'fwhm')


flame1 = pe.MapNode(interface=fsl.FLAMEO(run_mode='flame1'), name="flame1",
                    iterfield=['cope_file','var_cope_file'])
flame1.inputs.mask_file=os.path.join(os.getenv('FSLDIR'),'data/standard/MNI152_T1_2mm_brain_mask.nii.gz')
#fixed_fx.connect(datasource_mask,'mask',flameo,'mask_file')




mixed_fx.connect(smooth_cope,'smoothed_file',flame1,'cope_file')
mixed_fx.connect(smooth_varcope,'smoothed_file',flame1,'var_cope_file')
mixed_fx.connect(smooth_cope,'smoothed_file',datasink,'smoothed_cope')


mixed_fx.connect(level3model,'design_mat',flame1,'design_file')
mixed_fx.connect(level3model,'design_con',flame1,'t_con_file')
mixed_fx.connect(level3model,'design_grp',flame1,'cov_split_file')


#mixed_fx.connect(flame1,'stats_dir',datasink,'stats')

# compute smoothness of residuals

smoothest=pe.Node(fsl.SmoothEstimate(),name='smoothest')
smoothest.inputs.dof=15
smoothest.inputs.mask_file=os.path.join(os.getenv('FSLDIR'),'data/standard/MNI152_T1_2mm_brain_mask.nii.gz')

def pickfirst(files):
    if isinstance(files, list):
        return files[0]
    else:
        return files


mixed_fx.connect(flame1,('res4d',pickfirst),smoothest,'residual_fit_file')

# clustering with gaussian random field theory

cluster=pe.Node(fsl.Cluster(),name='cluster')
cluster.inputs.threshold = 3.0
cluster.inputs.pthreshold=0.05
cluster.inputs.out_threshold_file='thresh_zstat1.nii.gz'
cluster.inputs.out_localmax_txt_file='localmax.txt'

mixed_fx.connect(flame1,('zstats',pickfirst),cluster,'in_file')
mixed_fx.connect(smoothest,'dlh',cluster,'dlh')
mixed_fx.connect(smoothest,'volume',cluster,'volume')

mixed_fx.connect(cluster,'threshold_file',datasink,'cluster.thresh_zstat')
mixed_fx.connect(cluster,'localmax_txt_file',datasink,'cluster.localmax')



mixed_fx.run()


/Users/poldrack/anaconda/envs/py3k/lib/python3.5/site-packages/nipype/interfaces/base.py:432: UserWarning: Input pthreshold requires inputs: dlh, volume
  warn(msg)
INFO:workflow:['check', 'execution', 'logging']
INFO:workflow:Running serially.
INFO:workflow:Executing node datasource_cope.a0.a1 in dir: /Users/poldrack/data_unsynced/fmri-handbook-2e-data/ds005/nipype_workdir/mixedfx/_copenum_2/_regtype_ants/datasource_cope
INFO:workflow:Executing node datasource_varcope.a0.a0 in dir: /Users/poldrack/data_unsynced/fmri-handbook-2e-data/ds005/nipype_workdir/mixedfx/_copenum_1/_regtype_ants/datasource_varcope
INFO:workflow:Executing node datasource_cope.a1.a0 in dir: /Users/poldrack/data_unsynced/fmri-handbook-2e-data/ds005/nipype_workdir/mixedfx/_copenum_1/_regtype_affine/datasource_cope
INFO:workflow:Executing node l3model in dir: /Users/poldrack/data_unsynced/fmri-handbook-2e-data/ds005/nipype_workdir/mixedfx/l3model
INFO:workflow:Collecting precomputed outputs
INFO:workflow:Executing n

### Nonparametric correction using randomise

Run this outside of nipype because interface is currenly not working properly

In [28]:
randomise_output_dir=os.path.join(config.data['ds005']['datadir'],'derivatives/randomise')
if not os.path.exists(randomise_output_dir):
    os.mkdir(randomise_output_dir)


for cope in range(1,5):
    for fwhm in [0,4,8,16,32]:
        for regtype in ['affine','ants']:
            copefile=os.path.join(config.data['ds005']['datadir'],
                    'derivatives/smoothed_cope/_copenum_%d/_regtype_%s/_fwhm_%d/cope1_merged_smooth.nii.gz'%(cope,regtype,fwhm))
            print(copefile)
            assert os.path.exists(copefile)
            outdir=os.path.join(randomise_output_dir,'_copenum_%d/_regtype_%s/_fwhm_%d'%(cope,regtype,fwhm))
            if not os.path.exists(outdir):
                os.makedirs(outdir)
            nperms=2500
            cmd='randomise -i %s -o "%s/randomise_" -c 3.00 -C 3.00 -n %d -1 -T -v 10'%(copefile,outdir,nperms)
            print(cmd)
            if not os.path.exists(os.path.join(outdir,'randomise__tstat1.nii.gz')):
                !{cmd}


/Users/poldrack/data/fmri-handbook-2e-data/ds005/derivatives/smoothed_cope/_copenum_1/_regtype_affine/_fwhm_0/cope1_merged_smooth.nii.gz
randomise -i /Users/poldrack/data/fmri-handbook-2e-data/ds005/derivatives/smoothed_cope/_copenum_1/_regtype_affine/_fwhm_0/cope1_merged_smooth.nii.gz -o "/Users/poldrack/data/fmri-handbook-2e-data/ds005/derivatives/randomise/_copenum_1/_regtype_affine/_fwhm_0/randomise_" -c 3.00 -C 3.00 -n 2500 -1 -T -v 10
randomise options: -i /Users/poldrack/data/fmri-handbook-2e-data/ds005/derivatives/smoothed_cope/_copenum_1/_regtype_affine/_fwhm_0/cope1_merged_smooth.nii.gz -o /Users/poldrack/data/fmri-handbook-2e-data/ds005/derivatives/randomise/_copenum_1/_regtype_affine/_fwhm_0/randomise_ -c 3.00 -C 3.00 -n 2500 -1 -T -v 10 
Loading Data: 
Data loaded
One-sample design detected; sign-flipping instead of permuting.
65536 sign-flips required for exhaustive test of t-test 1
Doing 2500 random permutations
Starting permutation 1 (Unpermuted data)
Starting permutati

## Statistical maps using linear registration